In [1]:
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [2]:
colunas = ["PAV","AMBIENTE","TIPO","COMPRIMENTO","LARGURA","ALTURA","ÁREA DE SALA",
"ÁREA DE PORTAS","ÁREA DE JANELAS","QUANT. PESSOAS","EQUIPAMENTOS ELETRICOS","MARCA",
"MODELO","POT. (KBTU)","ETIQUETA","CEE","QUANT.","NAN"
]
df = pd.read_excel("CALCULO LUMINOTÉCNICO.xlsx",sheet_name=0,names=colunas)

In [3]:
df[ (df["ALTURA"].notnull()) & (df["ALTURA"]!="PLANTA CAD")].reset_index(drop=True).iloc[1:,:] # REMOVE LINHAS QUE NÃO TEM ALTURA e Remove a primeira linha que é parte do cabeçalho

,PAV,AMBIENTE,TIPO,COMPRIMENTO,LARGURA,ALTURA,ÁREA DE SALA,ÁREA DE PORTAS,ÁREA DE JANELAS,QUANT. PESSOAS,EQUIPAMENTOS ELETRICOS,MARCA,MODELO,POT. (KBTU),ETIQUETA,CEE,QUANT.,NAN
1,1.61,732,0.7,500,18740.259740,0.55,2,900,1800,10.411255,10.0,20,18.0,CAA01-E216,4000.0,180,NaN,B OU +
2,1.61,732,0.7,500,18571.428571,0.55,2,900,1800,10.317460,10.0,20,18.0,CAA01-E216,4000.0,180,NaN,B OU +
3,1.61,732,0.7,500,40870.129870,0.55,2,900,1800,22.705628,22.0,44,18.0,CAA01-E216,4000.0,396,NaN,B OU +
4,1.61,732,0.7,200,1994.805195,0.55,2,900,1800,1.108225,1.0,2,18.0,CAA01-E216,4000.0,18,NaN,B OU +
5,1.61,732,0.7,500,58935.064935,0.55,2,900,1800,32.741703,32.0,64,18.0,CAA01-E216,4000.0,576,NaN,B OU +
6,1.61,732,0.7,300,25527.272727,0.55,2,900,1800,14.181818,14.0,28,18.0,CAA01-E216,4000.0,252,NaN,B OU +
7,1.61,732,0.7,100,3480.519481,0.55,2,900,1800,1.933622,2.0,4,18.0,CAA01-E216,4000.0,36,NaN,B OU +
8,1.61,732,0.7,100,6864.935065,0.55,2,900,1800,3.813853,4.0,8,18.0,CAA01-E216,4000.0,72,NaN,B OU +
9,1.61,732,0.7,500,97350.649351,0.55,2,1850,3700,26.310986,26.0,52,36.0,CAA01-E232,4000.0,936,NaN,B OU +
10,1.61,732,0.7,200,32862.337662,0.55,2,900,1800,18.256854,18.0,36,18.0,CAA01-E216,4000.0,324,NaN,B OU +


In [4]:
dfLista = df.values.tolist() # Transforma o dataframe em lista e remove o cabeçalho
dfLista.pop(0)
dfListaAntiga = list(filter(lambda x: True 
if (x[6] != "PLANTA CAD" 
and str(x[3])!="nan" and str(x[3])!="-" 
and str(x[4])!="nan" and str(x[4])!="-" and str(x[4])!="10,87/9"
and str(x[5])!="nan" and str(x[5])!="-" 
and str(x[6])!="nan" and str(x[6])!="-" 
and str(x[0]!="nan") and str(x[1]!="nan")) 
else False,dfLista))#remove campos onde não alura da sala	

dfListaNova = []


print(df.columns)
for x in dfListaAntiga:
    print(x)
    dfListaNova.append(x)


Index(['PAV', 'AMBIENTE', 'TIPO', 'COMPRIMENTO', 'LARGURA', 'ALTURA',
       'ÁREA DE SALA', 'ÁREA DE PORTAS', 'ÁREA DE JANELAS', 'QUANT. PESSOAS',
       'EQUIPAMENTOS ELETRICOS', 'MARCA', 'MODELO', 'POT. (KBTU)', 'ETIQUETA',
       'CEE', 'QUANT.', 'NAN'],
      dtype='object')
[1.61, 732, 0.7, 500, 18740.25974025974, 0.55, 2, 900, 1800, 10.41125541125541, 10.0, 20, 18.0, 'CAA01-E216', 4000.0, 180, nan, 'B OU +']
[1.61, 732, 0.7, 500, 18571.428571428572, 0.55, 2, 900, 1800, 10.317460317460318, 10.0, 20, 18.0, 'CAA01-E216', 4000.0, 180, nan, 'B OU +']
[1.61, 732, 0.7, 500, 40870.129870129866, 0.55, 2, 900, 1800, 22.705627705627702, 22.0, 44, 18.0, 'CAA01-E216', 4000.0, 396, nan, 'B OU +']
[1.61, 732, 0.7, 200, 1994.8051948051948, 0.55, 2, 900, 1800, 1.1082251082251082, 1.0, 2, 18.0, 'CAA01-E216', 4000.0, 18, nan, 'B OU +']
[1.61, 732, 0.7, 500, 58935.06493506493, 0.55, 2, 900, 1800, 32.74170274170274, 32.0, 64, 18.0, 'CAA01-E216', 4000.0, 576, nan, 'B OU +']
[1.61, 732, 0.7, 300, 2552

In [5]:
dfNovo = pd.DataFrame(dfListaNova,columns=colunas)
#dfNovo = dfNovo[dfNovo["PAV"].notnull() & dfNovo["AMBIENTE"].notnull()]
dfNovo

,PAV,AMBIENTE,TIPO,COMPRIMENTO,LARGURA,ALTURA,ÁREA DE SALA,ÁREA DE PORTAS,ÁREA DE JANELAS,QUANT. PESSOAS,EQUIPAMENTOS ELETRICOS,MARCA,MODELO,POT. (KBTU),ETIQUETA,CEE,QUANT.,NAN
0,1.61,732,0.7,500,18740.259740,0.55,2,900,1800,10.411255,10.0,20,18.0,CAA01-E216,4000.0,180,NaN,B OU +
1,1.61,732,0.7,500,18571.428571,0.55,2,900,1800,10.317460,10.0,20,18.0,CAA01-E216,4000.0,180,NaN,B OU +
2,1.61,732,0.7,500,40870.129870,0.55,2,900,1800,22.705628,22.0,44,18.0,CAA01-E216,4000.0,396,NaN,B OU +
3,1.61,732,0.7,200,1994.805195,0.55,2,900,1800,1.108225,1.0,2,18.0,CAA01-E216,4000.0,18,NaN,B OU +
4,1.61,732,0.7,500,58935.064935,0.55,2,900,1800,32.741703,32.0,64,18.0,CAA01-E216,4000.0,576,NaN,B OU +
5,1.61,732,0.7,300,25527.272727,0.55,2,900,1800,14.181818,14.0,28,18.0,CAA01-E216,4000.0,252,NaN,B OU +
6,1.61,732,0.7,100,3480.519481,0.55,2,900,1800,1.933622,2.0,4,18.0,CAA01-E216,4000.0,36,NaN,B OU +
7,1.61,732,0.7,100,6864.935065,0.55,2,900,1800,3.813853,4.0,8,18.0,CAA01-E216,4000.0,72,NaN,B OU +
8,1.61,732,0.7,500,97350.649351,0.55,2,1850,3700,26.310986,26.0,52,36.0,CAA01-E232,4000.0,936,NaN,B OU +
9,1.61,732,0.7,200,32862.337662,0.55,2,900,1800,18.256854,18.0,36,18.0,CAA01-E216,4000.0,324,NaN,B OU +


In [6]:
dfNovo = dfNovo[["PAV","AMBIENTE","COMPRIMENTO","LARGURA","ALTURA"]]

In [7]:

colunas2 = ["PAV","AMBIENTE","TIPO","LÂMPADAS","MODELO","POT. (W)","QUANT.","POTÊNCIA","QUANT.","MODELO","LÂMPADAS/LUMINÁRIA","QUANT2.","LUZ NATURAL","COMANDO P/JANELA","COMANDO INDIVIDUAL","NAN"]
df2 = pd.read_excel("CALCULO LUMINOTÉCNICO.xlsx",sheet_name=1,names=colunas2)
df2 = df2[df2["POT. (W)"].notnull() & (df2["POT. (W)"]!="SI")].reset_index(drop=True).iloc[1:,:] #remove colunas onde não tem potencia e remove a primeira linha que é parte do cabeçalho
df2 = df2[["PAV","AMBIENTE","POT. (W)","QUANT."]]

ValueError: Number of passed names did not match number of header fields in the file (sheet: 1)

In [9]:
dfGeral = dfNovo.merge(df2,how="inner", left_on=["PAV","AMBIENTE"], right_on=["PAV","AMBIENTE"])

NameError: name 'df2' is not defined

In [ ]:
campos = {"PAV":"pavimento","AMBIENTE":"ambiente","COMPRIMENTO":"comprimento","LARGURA":"largura","ALTURA":"distanciaPlanoDeTrabalhoTeto","POT. (W)":"potenciaLampada","QUANT.":"qntdLampadas"}
dfGeral = dfGeral.rename(columns=campos)
dfGeral.drop_duplicates(inplace=True)
dfGeral

### Geração do PDF de Qr-Code

In [8]:
import requests
from fpdf import FPDF
import os
# save FPDF() class into a
# variable pdf
pdf = FPDF()

# set style and size of font
	# that you want in the pdf
pdf.set_font("Arial", size = 15)



for index,i in enumerate(dfGeral.values.tolist()):
    pavimento = i[0]

    ambiente = i[1]

    pdf.add_page() # add a page
    
	# create a cell
    pdf.cell(w = 0, h = 100, txt = f"PAV. {pavimento} - {ambiente}",
			ln = 1, align = 'C')

    

    
    response = requests.get(f"https://us-central1-eficiencia-energetica---jf.cloudfunctions.net/api/ambientesQrCode?pavimento={pavimento}&ambiente={ambiente}").json()
    qrCodeAmbiente = response["qrCodeAmbiente"]
    response2 = requests.get(qrCodeAmbiente)

    imgDir = f"qrCodeAmbiente{index}.png"

    file = open(imgDir, "wb")
    file.write(response2.content)
    file.close()

    #BytesIO(response2.content)
    pdf.image(imgDir, x = pdf.w/2-50/2, y = pdf.h/2-50/2, w = 50, h = 50)

    os.remove(imgDir)#remove a imagem da pasta


pdf.output("outros/qrCodesAmbientes.pdf")


NameError: name 'dfGeral' is not defined

### Carga no Banco de Dados

In [ ]:
import requests
for i in dfGeral.values.tolist():
    json = {"pavimento":str(i[0]),"ambiente":str(i[1]),"comprimento":i[2],"largura":i[3],"distanciaPlanoDeTrabalhoTeto":i[4],"potenciaLampada":i[5],"qntdLampadas":i[6]}
    response = requests.post(f"https://us-central1-eficiencia-energetica---jf.cloudfunctions.net/api/ambientes",json=json)
    print(f"PAV - {i[0]} Ambiente - {i[1]} ====> status:{response.status_code}")